In [2]:
pip install torch torchvision pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 8.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

  Using cached opencv-python-4.11.0.86.tar.gz (95.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for opencv-python: filename=opencv_python-4.11.0.86-cp312-cp312-macosx_12_0_arm64.whl size=13160473 sha256=2fd9ddd985afa58f1ae970a216d14ea18c1f0453a049bca414762f74c42c0c3b
  Stored in directory: /Users/keito/Library/Caches/pip/wheels/be/bd/d5/425eca52f204ab4b1ad7ac23c79e7a0458ee178056e4350265
Successfully built opencv-python
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install opencv-contrib-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 MB 2.4 MB/s eta 0:00:0000:0100:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for opencv-contrib-python: filename=opencv_contrib_python-4.11.0.86-cp312-cp312-macosx_12_0_arm64.whl size=18939252 sha256=2461f1024b637ee8ebd4c6ea5492e25ddd1651af8e3ec8d8048e63b0fa990712
  Stored in directory: /Users/keito/Library/Caches/pip/wheels/fb/4e/b6/7f958a2039bdae6b0da86e3e497e6d5ec176c65227e2494df8
Successfully built opencv-contrib-python
Note: you may need to restart the kernel to use updated packages.


In [23]:
from pycocotools.coco import COCO
import torch
from torch.utils.data import Dataset
import cv2
import os
import torchvision.transforms.functional as F


class COCODataset(Dataset):
    def __init__(self, img_dir, ann_path, transforms=None):
        self.coco = COCO(ann_path)
        self.img_dir = img_dir
        self.transforms = transforms
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        path = self.coco.loadImgs(img_id)[0]['file_name']
    
        img = cv2.imread(os.path.join(self.img_dir, path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = F.to_tensor(img)
    
        boxes = []
        labels = []
    
        for ann in anns:
            x, y, w, h = ann['bbox']
            # Пропускаем боксы с нулевой шириной/высотой
            if w < 1 or h < 1:
                continue
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
    
        if len(boxes) == 0:
            return None
    
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
    
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([img_id]),
        }
    
        return img, target


    def __len__(self):
        return len(self.ids)


In [19]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    # Загружаем предобученную модель
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # Меняем классификатор под наше число классов
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


In [71]:
from torch.utils.data import DataLoader
import torch.optim as optim


def collate_fn(batch):
    batch = [b for b in batch if b is not None]  # удаляем пустые
    if len(batch) == 0:
        return None, None  # пустой батч, пропустим
    return tuple(zip(*batch))


# Путь к папке с изображениями и аннотациям
dataset = COCODataset("/Users/keito/Downloads/Wind Turbine damage/train/", "/Users/keito/Downloads/Wind Turbine damage/train/_annotations.coco.json")
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes=6)
model.to(device)

# Оптимизатор
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Цикл обучения (упрощён)
model.train()
for epoch in range(5):
    for images, targets in data_loader:
            if images is None or targets is None:
                continue  # пропускаем пустой батч
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
    
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

    print(f"Эпоха {epoch}, потери: {losses.item()}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Эпоха 0, потери: 0.8509682416915894
Эпоха 1, потери: 0.9081409573554993
Эпоха 2, потери: 0.6725689172744751
Эпоха 3, потери: 0.595337450504303
Эпоха 4, потери: 0.9318835735321045


In [74]:
torch.save(model.state_dict(), "model.pth")

In [82]:
# я хочу теперь обучить модель на валид картинках, чтобы наверняка

In [86]:
model = get_model(num_classes=6)
model.load_state_dict(torch.load("model.pth"))  # загружаем веса из предыдущего обучения
model.to(device)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [88]:
valid_dataset = COCODataset(
    "/Users/keito/Downloads/Wind Turbine damage/valid/",
    "/Users/keito/Downloads/Wind Turbine damage/valid/_annotations.coco.json"
)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [90]:
model.train()
for epoch in range(2):  # можно 1–3 эпохи, не больше
    for images, targets in valid_loader:
        if images is None or targets is None:
            continue  # пропускаем пустые

        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"[VALID] Эпоха {epoch}, потери: {losses.item():.4f}")

[VALID] Эпоха 0, потери: 0.2263
[VALID] Эпоха 1, потери: 0.3053


In [92]:
# и потом сохранить новую модель
torch.save(model.state_dict(), "model_finetuned.pth")
print("Сохранено: model_finetuned.pth")

Сохранено: model_finetuned.pth


In [76]:
import pandas as pd

test_dataset = COCODataset("/Users/keito/Downloads/Wind Turbine damage/valid/", "/Users/keito/Downloads/Wind Turbine damage/valid/_annotations.coco.json")  # Если есть аннотации
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None, None
    return tuple(zip(*batch))

data_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn)

submission = []

model.eval()
with torch.no_grad():
    for images, image_ids in data_loader:
        if images is None:
            continue

        images = [img.to(device) for img in images]
        outputs = model(images)

        for output, image_id in zip(outputs, image_ids):
            boxes = output["boxes"].cpu().numpy()
            labels = output["labels"].cpu().numpy()
            scores = output["scores"].cpu().numpy()

            for box, label, score in zip(boxes, labels, scores):
                if score < 0.5:
                    continue
                x1, y1, x2, y2 = box
                submission.append([
                    image_id,  # строка с ID изображения
                    int(label),
                    float(score),
                    float(x1), float(y1), float(x2), float(y2)
                ])



df = pd.DataFrame(submission)
df.to_csv("submission.csv", index=False)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


KeyboardInterrupt: 

In [94]:
import os
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image
import pandas as pd

# === Настройки ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_FOLDER = "/Users/keito/Downloads/Wind Turbine damage/test/"
MODEL_PATH = "model.pth"  # путь к сохраненной модели
SUBMISSION_PATH = "submission.csv"

# === Функция трансформации изображений ===
transform = T.Compose([
    T.ToTensor()
])

# === Загрузка модели ===
def get_model(num_classes=6):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model()
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# === Инференс и формирование submission ===
submission = []

image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.endswith(".jpg") or f.endswith(".png")]

for image_name in image_files:
    image_path = os.path.join(IMAGE_FOLDER, image_name)
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).to(DEVICE)

    with torch.no_grad():
        outputs = model([img_tensor])[0]

    pred_str = ""
    for box, label, score in zip(outputs['boxes'], outputs['labels'], outputs['scores']):
        if score < 0.3:
            continue
        x_min, y_min, x_max, y_max = box
        pred_str += f"{label.item()} {score.item():.4f} {x_min.item():.1f} {y_min.item():.1f} {x_max.item():.1f} {y_max.item():.1f} "

    submission.append({
        "image_id": image_name,
        "objects": pred_str.strip()
    })

# === Сохранение в CSV ===
df = pd.DataFrame(submission)
df.to_csv(SUBMISSION_PATH, index=False)
print(f"Saved submission to {SUBMISSION_PATH}")


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Saved submission to submission.csv


In [96]:
import pandas as pd
import json

# Классы
class_names = {
    1: "corrosion",
    2: "lightning",
    3: "lightning receptor",
    4: "missing teeth",
    5: "patch"
}

def parse_prediction_string(pred_str):
    if pd.isna(pred_str) or not isinstance(pred_str, str):
        return []

    numbers = list(map(float, pred_str.strip().split()))

    # Проверка на класс 
    # for i in range(0, len(numbers), 6):
    #     cls = int(numbers[i])
    #     if cls >= 5:
    #         return []
    predictions = []

    for i in range(0, len(numbers), 6):
        cls, score, x_min, y_min, x_max, y_max = numbers[i:i+6]

        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min

        predictions.append([
            class_names[int(cls)],
            x_center, y_center,
            width, height,
            score
        ])

    return predictions

# Загрузка исходного CSV
df = pd.read_csv("/Users/keito/rsnn/submission.csv")

# Преобразование
df["objects"] = df["objects"].apply(lambda x: json.dumps(parse_prediction_string(x)))

# Сохранение в нужном формате
df.to_csv("submission_converted13.csv", index=False)